In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import BernoulliNB

In [2]:
test = pd.read_csv('include/test.csv')
train = pd.read_csv('include/train.csv').dropna()

In [3]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if str(big_string).find(substring) != -1:
            return substring
    return ''

In [4]:
train_reformated = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
train_reformated['Deck'] = train['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
train_reformated['Family_Size'] = train_reformated['SibSp'] + train_reformated['Parch']
train_reformated['Fare_Per_Person'] = train_reformated['Fare'] /(train_reformated['Family_Size'] + 1)

train_reformated['Sex'] =  LabelEncoder().fit_transform(train_reformated['Sex'])
train_reformated['Embarked'] = LabelEncoder().fit_transform(train_reformated['Embarked'])
train_reformated['Deck'] = LabelEncoder().fit_transform(train_reformated['Deck'])
train_reformated.drop(['Fare', 'Embarked', 'Family_Size', 'Parch', 'SibSp', 'Deck'], axis=1, inplace=True)
print(train_reformated.head())

    Survived  Pclass  Sex   Age  Fare_Per_Person
1          1       1    0  38.0        35.641650
3          1       1    0  35.0        26.550000
6          0       1    1  54.0        51.862500
10         1       3    0   4.0         5.566667
11         1       1    0  58.0        26.550000


In [5]:
clf = BernoulliNB()
clf.fit(train_reformated.loc[:, train_reformated.columns != 'Survived'], train_reformated['Survived'])

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [6]:
test_passengerId = test['PassengerId']
test_reformated = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
test_reformated['Deck'] = test['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
test_reformated['Family_Size'] = test_reformated['SibSp'] + test_reformated['Parch']
test_reformated['Fare_Per_Person'] = test_reformated['Fare'] /(test_reformated['Family_Size'] + 1)

test_reformated['Sex'] =  LabelEncoder().fit_transform(test_reformated['Sex'])
test_reformated['Embarked'] = LabelEncoder().fit_transform(test_reformated['Embarked'])
test_reformated['Deck'] = LabelEncoder().fit_transform(test_reformated['Deck'])
test_reformated.drop(['SibSp', 'Parch', 'Fare', 'Embarked', 'Family_Size', 'Deck'], axis=1, inplace=True)
print(test_reformated.head())

   Pclass  Sex   Age  Fare_Per_Person
0       3    1  34.5         7.829200
1       3    0  47.0         3.500000
2       2    1  62.0         9.687500
3       3    1  27.0         8.662500
4       3    0  22.0         4.095833


In [7]:
print(test_reformated.columns[test_reformated.isnull().any()].tolist())
test_reformated = test_reformated.fillna(test_reformated.mean())

test_predictions = clf.predict(test_reformated).reshape(-1, 1)

combined = np.column_stack((test_passengerId, test_predictions))
df = pd.DataFrame(combined, columns=["PassengerId", "Survived"])

['Age', 'Fare_Per_Person']


In [8]:
df.to_csv('csv/answers10.csv',index=False)

In [9]:
print(test.head)

<bound method NDFrame.head of      PassengerId  Pclass                                               Name  \
0            892       3                                   Kelly, Mr. James   
1            893       3                   Wilkes, Mrs. James (Ellen Needs)   
2            894       2                          Myles, Mr. Thomas Francis   
3            895       3                                   Wirz, Mr. Albert   
4            896       3       Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
5            897       3                         Svensson, Mr. Johan Cervin   
6            898       3                               Connolly, Miss. Kate   
7            899       2                       Caldwell, Mr. Albert Francis   
8            900       3          Abrahim, Mrs. Joseph (Sophie Halaut Easu)   
9            901       3                            Davies, Mr. John Samuel   
10           902       3                                   Ilieff, Mr. Ylio   
11           903      